In [1]:
import pandas as pd
import numpy as np
import gensim
from random import sample

In [2]:
# Load data
train_chal= pd.read_csv('train.csv')
test_chal = pd.read_csv('test.csv')
chall_details= pd.read_csv('challenge_data.csv')

In [3]:
train=train_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [4]:
test=test_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [5]:
set(train.user_id).intersection(test.user_id
                               )

set()

In [6]:
train.shape, test.shape

((69532, 2), (39732, 2))

In [7]:
sentence_list=pd.concat((train.challenge,test.challenge)).apply(lambda x: x.split())
# sentence_list= train.challenge.apply(lambda x: x.split())

In [225]:
# train word2vec model to learn embeddings
neg=15
itera=100
window=5
size=500
print 'doing_'+str(neg)+'_'+str(window)+'_'+str(itera)
model=gensim.models.Word2Vec(sentence_list,size=size,window=window,workers=8,min_count=0, hs=1,iter=itera,sg=1)
model.save('word2vec_solved_iter__noshuf_5window_'+ str(itera) + '_size_' + str(size))

doing_15_5_100


In [9]:
# model= gensim.models.Word2Vec.load('word2vec_solved_iter__noshuf_5window_100_size_500')

In [236]:
model.most_similar('CI23855')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('CI23663', 0.3396339416503906),
 ('CI23975', 0.32559213042259216),
 ('CI23933', 0.31812340021133423),
 ('CI25135', 0.29852694272994995),
 ('CI24915', 0.283422589302063),
 ('CI23913', 0.2829606831073761),
 ('CI25125', 0.2683682441711426),
 ('CI23714', 0.2660852074623108),
 ('CI24527', 0.2611076831817627),
 ('CI24958', 0.2601049542427063)]

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Merge, TimeDistributed, concatenate, Bidirectional, Masking, RepeatVector
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers.convolutional import Convolution1D, MaxPooling1D


Using TensorFlow backend.


In [11]:
pretrained_weights = model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

('Result embedding shape:', (5502, 500))


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [12]:
def word2idx(word):
  return model.wv.vocab[word].index
def idx2word(idx):
  return model.wv.index2word[idx]

In [13]:
# generate int sequences for challenge sentences
train_seq=np.array(train.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())
test_seq=np.array(test.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())

In [29]:
xtrain=train_seq[:,:10]
ytrain= train_seq[:,10]
xtrain.shape, ytrain.shape

((69532, 10), (69532,))

In [30]:
# add 11th,12th,13th challenge to outcome
xtrain= np.concatenate((xtrain,xtrain,xtrain),axis=0)
ytrain= np.concatenate((train_seq[:,10],train_seq[:,11],train_seq[:,12]))

In [79]:
# define nn model
emdedding_size=500
vocab_size=5502
keras_model2 = Sequential()
keras_model2.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))
keras_model2.add(GRU(units=emdedding_size))
keras_model2.add(Dense(units=vocab_size))

keras_model2.add(Activation('softmax'))
keras_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [41]:
sample_weight=np.concatenate((np.ones(train.shape[0])*3,np.ones(train.shape[0])*2,np.ones(train.shape[0])))

In [ ]:
# fit differnt samples of data and then full data
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1)

Epoch 1/10
  1024/208596 [..............................] - ETA: 938s - loss: 17.0322 

val loss= 5.0348, 5.07

In [77]:
import pickle
pickle.dump(keras_model2.to_json(),open('gru4.pkl','w'))

In [78]:
import h5py
import keras
# from importlib import reload
# import keras
keras_model2.save_weights('gru4.weights')

In [422]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return preds.argsort()[-3:][::-1]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
#     print max(probas[0])
    retar= probas[0].argsort()[-3:][::-1]
    return retar

def generate_next(text):
    global count
    count+=1
    if count%100==0:
        print count
    word_idxs = [word2idx(word) for word in text.split()]
    prediction = keras_model2.predict(x=np.array(word_idxs))
#     print prediction
    idxl = sample(prediction[-1], temperature=0.0)
#     word_idxs.e(idx)
#     print idxl
    return [idx2word(idx) for idx in idxl]



In [390]:
test.challenge.values[0]

'CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530'

In [423]:
generate_next('CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530')

['CI26215', 'CI26903', 'CI26221']

In [18]:
test_inds=np.array(test.challenge.apply(lambda x: np.array([word2idx(word) for word in x.split()])).tolist())

In [19]:
test_inds.shape, xtrain.shape

((39732, 10), (69532, 10))

In [73]:
# make predictions
nnpred=keras_model2.predict(test_inds)

In [74]:
nnpred.shape

(39732, 5502)

In [75]:
nnpred2= pd.Series(nnpred.argsort(axis=1)[:,-3:].tolist())

In [76]:
# store to csv
count=0
test['pred']=nnpred2.apply(lambda x: [idx2word(y) for y in x[::-1]])
test_sub= test.copy()
test_sub=pd.concat((test_sub,test_sub,test_sub))
test_sub=test_sub.sort_values('user_id').reset_index(drop=True)
test_sub['seq']=test_sub.groupby('user_id').cumcount()
test_sub.seq= test_sub.seq+11
test_sub['user_sequence']= test_sub.user_id.astype('str') + '_' + test_sub.seq.astype('str')
test_sub['challenge']=test_sub.apply(lambda row: row['pred'][row['seq']-11], axis=1)
test_sub[['user_sequence','challenge']].to_csv('ksubmission.csv',index=False)